In [1]:
!pip install google-generativeai pandas openpyxl

In [3]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [4]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.47
    Uninstalling langchain-core-0.3.47:
      Successfully uninstalled langchain-core-0.3.47
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


In [32]:
import json
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import google.generativeai as genai
import pandas as pd
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

In [27]:
# Configure Google Gemini API
genai.configure(api_key="YOUR_API_KEY_HERE")

# Define the Quiz Structure
class Quiz(BaseModel):
    question: str = Field(description="Question for the quiz")
    answer_1: str = Field(description="One of the answer choices")
    answer_2: str = Field(description="One of the answer choices")
    answer_3: str = Field(description="One of the answer choices")
    answer_4: str = Field(description="One of the answer choices")
    correct_answer: str = Field(description="One or more correct answer choices (1, 2, 3, 4) separated by comma")
    time_limit: int = Field(description="Time limit in seconds")
    extra_info: str = Field(description="Additional content such as code or equations")

# Parser for structured output
parser = JsonOutputParser(pydantic_object=Quiz)

In [28]:
def generate_quiz_questions(topic, num_questions):
    """Generate quiz questions using Gemini API."""
    model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

    prompt_template = PromptTemplate(
        template="""Generate {num_questions} multiple-choice quiz questions on {topic}.
        Format each question as a JSON object with:
        - "question": The question text (max 95 chars)
        - "extra_info": Code or equation related to the question (if applicable), formatted as a Markdown code block (```swift ... ```)
        - "answer_1", "answer_2", "answer_3", "answer_4": Four answer choices (max 60 chars each)
        - "correct_answers": One or more correct answer choices (1, 2, 3, or 4) separated by comma
        - "time_limit": Time limit for answering in seconds

        Wrap code in triple backticks to preserve formatting.

        Output should be a JSON array of {num_questions} questions.
        """,
        input_variables=["topic", "num_questions"]
    )

    chain = prompt_template | model
    response = chain.invoke({"topic": topic, "num_questions": num_questions}).content

    # Attempt to parse JSON with relaxed rules
    try:
        quiz_data = json.loads(response, strict=False)
    except json.JSONDecodeError:
        print("Error parsing JSON response. Cleaning output...")
        response = response.strip("```json").strip("```").strip()
        quiz_data = json.loads(response, strict=False)

    return quiz_data


In [29]:
def save_to_excel(quiz_data, filename="quiz.xlsx"):
    """Save quiz questions to an Excel file."""
    df = pd.DataFrame(quiz_data)
    df.to_excel(filename, index=False)
    print(f"Quiz saved to {filename}")

In [35]:
def save_text_as_image(text, folder="quiz_images", filename="extra_info.jpg"):
    """Save code or equations as an image inside a folder."""
    os.makedirs(folder, exist_ok=True)  # Create folder if not exists

    font = ImageFont.load_default()
    img_width = 800
    img_height = max(50, len(text.split("\n")) * 20)
    img = Image.new("RGB", (img_width, img_height), "white")
    draw = ImageDraw.Draw(img)

    # Write text on image
    draw.text((10, 10), text, fill="black", font=font)

    # Save image inside the folder
    file_path = os.path.join(folder, filename)
    img.save(file_path)
    print(f"Saved additional info as {file_path}")

def zip_images(folder_name, zip_filename):
    """Zip the quiz_images folder for easy download."""
    shutil.make_archive(zip_filename, 'zip', folder_name)
    print(f"Images zipped as {zip_filename}")

In [36]:
def main():
    topic = "Swift programming, focusing on functions, loops, and conditionals." #Set topic for the quiz
    num_questions = 5 #Set the number of questions

    # Generate quiz
    quiz_questions = generate_quiz_questions(topic, num_questions)

    print(quiz_questions)

    # Convert to JSON format
    quiz_data = []
    for q in quiz_questions:
        quiz_data.append({
            "Question": q['question'],
            "Answer 1": q['answer_1'],
            "Answer 2": q['answer_2'],
            "Answer 3": q['answer_3'],
            "Answer 4": q['answer_4'],
            "Correct Answers": q['correct_answers'],
            "Time Limit": q['time_limit']
        })

        folder_name = "quiz_images"
        zip_filename = "quiz_images"

        # Save extra info as image if available
        if q['extra_info']:
            save_text_as_image(q['extra_info'], folder_name, filename=f"extra_info_{quiz_questions.index(q)+1}.jpg")

    # Save to Excel
    save_to_excel(quiz_data)

    #Zip images folder
    zip_images(folder_name, zip_filename)

if __name__ == "__main__":
    main()

Error parsing JSON response. Cleaning output...
[{'question': 'What is the output of the following Swift code?', 'extra_info': '```swift\nfunc calculate(x: Int, y: Int) -> Int {\n  if x > y {\n    return x - y\n  } else {\n    return y + x\n  }\n}\n\nprint(calculate(x: 5, y: 3))\n```', 'answer_1': '2', 'answer_2': '8', 'answer_3': '3', 'answer_4': '5', 'correct_answers': '1', 'time_limit': 45}, {'question': 'Which keyword is used to define a function in Swift?', 'extra_info': None, 'answer_1': 'class', 'answer_2': 'func', 'answer_3': 'var', 'answer_4': 'let', 'correct_answers': '2', 'time_limit': 30}, {'question': 'What is the output of the following loop?', 'extra_info': '```swift\nfor i in 1...3 {\n  print(i)\n}\n```', 'answer_1': '1 2', 'answer_2': '1 2 3', 'answer_3': '0 1 2', 'answer_4': '0 1 2 3', 'correct_answers': '2', 'time_limit': 40}, {'question': 'Which conditional statement executes if the condition is false?', 'extra_info': None, 'answer_1': 'if', 'answer_2': 'else', 'ans